In [1]:
import requests
from bs4 import BeautifulSoup
import urllib

In [2]:
# スクレイピングするページ数
max_page = 100
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
base_url = "https://suumo.jp"
# URLにページ番号を追加するプレースホルダを含む
relative_url = "/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13112&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=&pn={}"

In [17]:
# リクエストがうまく行かないパターンを回避するためのやり直し
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

data_samples = []
unique_samples = {}

# ヘッダー行を定義
headers = ["セグメント", "建物名", "住所", "最寄り駅1", "最寄り駅2", "最寄り駅3", "築年数", "最大階数", "階", "家賃", "管理費", "敷金", "礼金", "間取り", "面積", "URL" ,"エリア", "家賃_正規化", "管理費_正規化", "敷金_正規化", "礼金_正規化"]

# スプレッドシートに書き込むためのデータリスト
data_samples = [headers]  # ヘッダー行を最初に追加

for page in range(1, max_page + 1):
    # 完全なURLを生成
    full_url = base_url + relative_url.format(page)
    soup = load_page(full_url)
    mother = soup.find_all(class_='cassetteitem')
    # ここに各ページからのデータを処理して追加するコードを書く


    for child in mother:
        # 建物情報
        segment = child.find(class_='ui-pct ui-pct--util1').text
        name = child.find(class_='cassetteitem_content-title').text
        address = child.find(class_='cassetteitem_detail-col1').text
        station_info = [item.text for item in child.find(class_='cassetteitem_detail-col2').find_all(class_='cassetteitem_detail-text')]
        year, maxfloor = [item.text for item in child.find(class_='cassetteitem_detail-col3').find_all('div')]

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            floor, rent_fee, maintenance_fee, deposit_fee, gratuity_fee, layout, area, url = '', '', '', '', '', '', '', ''

            # 部屋情報の取得
            for id_, grandchild in enumerate(room.find_all('td')):
                if id_ == 2:
                    floor = grandchild.text.strip()
                elif id_ == 3:
                    rent_fee = grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text
                    maintenance_fee = grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text
                elif id_ == 4:
                    deposit_fee = grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text
                    gratuity_fee = grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text
                elif id_ == 5:
                    layout = grandchild.find(class_='cassetteitem_madori').text
                    area = grandchild.find(class_='cassetteitem_menseki').text
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    url = urllib.parse.urljoin(url, get_url)

                    data_sample = [segment, name, address] + station_info + [year, maxfloor, floor, rent_fee, maintenance_fee, deposit_fee, gratuity_fee, layout, area, url]

                    # 住所、家賃、敷金、礼金、管理費の正規化
                    import pandas as pd
                    import re
                    import math

                    # 仮のデータフレームを作成
                    df = pd.DataFrame(data_samples[1:], columns=headers)

                    # 住所の正規化関数
                    def normalize_address(address):
                        # ここに住所を正規化するためのコードを書く
                        # 以下は例として「都」以降「区」までの部分を抽出する正規表現を使用しています
                        pattern = re.compile(r'都(.*?区)')
                        match = pattern.search(address)
                        if match:
                            return match.group(1)
                        else:
                            return address
                    def convert_fee(fee):
                        match = re.search(r'(\d+(?:\.\d+)?)万円', fee)
                        if match:
                            return int(float(match.group(1)) * 10000)
                        elif re.search(r'(\d+)円', fee):
                            return int(re.search(r'(\d+)円', fee).group(1))
                        else:
                            return None

                    data_sample.append(normalize_address(address))
                    data_sample.append(convert_fee(rent_fee))
                    data_sample.append(convert_fee(maintenance_fee))
                    data_sample.append(convert_fee(deposit_fee))
                    data_sample.append(convert_fee(gratuity_fee))

                    dedup_key = (data_sample[2], data_sample[6], data_sample[7], data_sample[8], data_sample[13], data_sample[14])

                    # 重複がなければ辞書に追加
                    if dedup_key not in unique_samples:
                        unique_samples[dedup_key] = data_sample
                        data_samples.append(data_sample)

In [18]:
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials

In [19]:
# Googleスプレッドシートの認証情報を取得
SCOPES = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = "tech0-step3_1_1-suumo-jsonkey.json"

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)
gs = gspread.authorize(credentials)

SPREADSHEET_KEY = "1rRtCz79tHmcBlmRGqbQDgwxgA_P8822bFhcb-k6MxQs"

workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("20240125")

# スプレッドシートに書き込むためのデータ
values = [headers] + list(unique_samples.values())

# スプレッドシートの1行目（A1セル）からデータを追加
worksheet.update("A1", values)

C:\Users\kurih\AppData\Local\Temp\ipykernel_7944\1426865199.py:17: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("A1", values)


{'spreadsheetId': '1rRtCz79tHmcBlmRGqbQDgwxgA_P8822bFhcb-k6MxQs',
 'updatedRange': "'20240125'!A1:U4482",
 'updatedRows': 4482,
 'updatedColumns': 21,
 'updatedCells': 90354}